In [161]:
# проверка работоспособности сервиса

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from datetime import datetime
from sqlalchemy import create_engine
import torch

In [212]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [213]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [215]:
user_data = pd.read_sql('SELECT * FROM public.user_data', con = engine)

def load_features() -> pd.DataFrame:
    feed_data = batch_load_sql("""
        SELECT *
        FROM public.feed_data
        WHERE action='like'
        limit 10000000""")
    return feed_data

preprocess_posts = pd.read_sql('SELECT * FROM public.kazakov_mta3669_features_dl', con = engine)


In [168]:
# Эта таблица нужна для того, чтобы вновь использовать человекочитаемые посты после применения модели.>>

posts = pd.read_sql('SELECT * FROM public.post_text_df', con = engine)
posts.set_index('post_id', inplace=True) 
# ---------------------- #

In [216]:
features_data = load_features() 

In [217]:
request = {
    'id': 2022,
    'time': datetime(2023, 5, 17),
    'limit': 5
}

In [218]:
def recommendations(id: int, time: datetime, limit: int):
    # Сначала удалим посты из выборки, которые пользователь уже видел
    user_view_posts = (features_data[features_data['user_id'] == id]
    [['post_id', 'timestamp']].drop_duplicates(subset=['post_id']).reset_index(drop=True)
                      )
    viewed_ids = user_view_posts['post_id'].tolist() 
    select_posts = preprocess_posts[~preprocess_posts['post_id'].isin(viewed_ids)]
    #return select_posts
    # определим фичи конкретного юзера
    user_init = user_data[user_data['user_id'] == id]
    # Добавляем их к табл. select_posts
    select_posts_use = select_posts.copy()
    user_dates = features_data[features_data['user_id'] == id]['timestamp']
    user_dates_mean = user_dates.mean()
    select_posts_use.insert(0, 'month', user_dates_mean.month)
    select_posts_use.insert(1, 'hour', user_dates_mean.hour)
    for col in user_init.columns:
        select_posts_use = select_posts_use.assign(**{col: user_init[col].iloc[0]})
    new_order = list(user_init.columns) + [col for col in select_posts_use.columns if col not in user_init.columns]
    select_posts_use = select_posts_use[new_order]
    # Создаем мультииндекс user_id / post_id
    #return select_posts_use
    select_posts_use = select_posts_use.set_index(['user_id', 'post_id'])
    select_posts_use.drop(['index'], axis = 1, inplace=True)
    loaded_model = CatBoostClassifier()
    loaded_model.load_model('catboost_model_mta3669.cbm')
    # Проверка
    expected_features = loaded_model.feature_names_
    #print("Ожидаемые признаки:", expected_features)
    #print("Фактические признаки:", select_posts_use.columns.tolist())
    select_posts_use['predict'] = loaded_model.predict_proba(select_posts_use)[:,1]
    #return select_posts_use
    typle_idx = select_posts_use['predict'].nlargest(limit).index.tolist()
    posts_idx = []
    for item in typle_idx:
        posts_idx.append(item[1])
    #Вывести должны мы список из словарей
    final_posts = []
    for idx in posts_idx:
        myDict = {
            'id':  idx,
            'text': posts.loc[idx, 'text'],
            'topic': posts.loc[idx, 'topic']
        }
        final_posts.append(myDict)
    return final_posts

In [219]:
output_function = recommendations(request['id'], request['time'], request['limit'])
#len(output_function)
output_function

[{'id': 1739,
  'text': 'Murray to make Cup history\n\nAndrew Murray will become Britains youngest-ever Davis Cup player after it was confirmed he will play in the doubles against Israel on Saturday.\n\nThe 17-year-old will play alongside fellow debutant David Sherwood against Israels Jonathan Erlich and Andy Ram. Murray will eclipse the record set by Roger Becker back in 1952. Greg Rusedski takes Tim Henmans place as first choice in the singles, while Alex Bogdanovic will play in the second singles clash. Rusedski will take on former world number 30 Harel Levy and Bogdanovic - who has previously played in two singles rubbers against Australia - will face Noam Okun.\n\nMurray is the brightest young hope in British tennis, after winning the US Open junior title last year and the BBC Young Sports Personality of the Year. British number one Tim Henman, who announced his Davis Cup retirement earlier this year, believes Britain can win the tie in Tel Aviv. Its going to be as really tough ma

In [104]:
user_data.os.unique()

array(['Android', 'iOS'], dtype=object)

In [ ]:
# 1. Прежде выведем какова средняя вероятность лайка у пользователя в просмотренных постах для дальнейшего добавления в target
    # 2. timestamp берем среднее или min?
    viewing_posts = features_data[features_data['user_id'] == id]['target'].count()
    like_posts = features_data[(features_data['user_id'] == id) & (features_data['target'] == 1)]['target'].count()
    like_frequency = like_posts / viewing_posts if viewing_posts > 0 else 0
    #return like_frequency
    user_dates = features_data[features_data['user_id'] == id]['timestamp']
    mean_date = user_dates.mean()
    min_date = user_dates.min()
    # добавляем из user_view_posts ['mean_date', 'like_frequency'] к табл select_posts_use 
    select_posts_use['target'] = like_frequency
    select_posts_use['min_date'] = min_date
    #return select_posts_use
    categorical_features = ['country', 'city', 'os', 'source']
    C = 0.006
    for col in categorical_features:
        if col in ['os', 'source']:
            if select_posts_use[col].nunique() == 2: 
                dummies = pd.get_dummies(select_posts_use[col], prefix=col, dtype=int, drop_first=True)
                select_posts_use = pd.concat([select_posts_use, dummies], axis=1)
                select_posts_use = select_posts_use.drop(col, axis=1)
            else:
                if col == 'os':
                    select_posts_use['os_iOS'] = (select_posts_use[col] == 'iOS').astype(int)
                elif col == 'source':
                    select_posts_use['source_organic'] = (select_posts_use[col] == 'organic').astype(int)
                select_posts_use = select_posts_use.drop(col, axis=1)
        else:
            means = select_posts_use.groupby(col)['target'].mean()
            noisy_means = means + C * np.random.randn(len(means))
            mapping_dict = noisy_means.to_dict()
            select_posts_use[col] = select_posts_use[col].map(mapping_dict)
    
    select_posts_use = select_posts_use.drop(['target'], axis = 1) 
    select_posts_use['day'] = select_posts_use['min_date'].dt.dayofweek
    select_posts_use['month'] = select_posts_use['min_date'].dt.month
    select_posts_use['hour'] = select_posts_use['min_date'].dt.hour
    select_posts_use = select_posts_use.drop(['min_date'],axis=1)
    #return select_posts_use.columns.tolist()
  
    #Загрузка модели
    loaded_model = CatBoostClassifier()
    loaded_model.load_model('catboost.cbm')
    # Проверка
    expected_features = loaded_model.feature_names_
    #print("Ожидаемые признаки:", expected_features)
    #print("Фактические признаки:", select_posts_use.columns.tolist())

    select_posts_use['predict'] = loaded_model.predict_proba(select_posts_use)[:,1]
    typle_idx = select_posts_use['predict'].nlargest(limit).index.tolist()
    posts_idx = []
    for item in typle_idx:
        posts_idx.append(item[1])
    #Вывести мы должны список из словарей
    final_posts = []
    for idx in posts_idx:
        myDict = {
            'id':  idx,
            'text': posts.loc[idx, 'text'],
            'topic': posts.loc[idx, 'topic']
        }
        final_posts.append(myDict)
    return final_posts
'''

In [ ]:
#preprocess_posts.describe()

In [44]:
features_data.head(2)

,timestamp,user_id,post_id,action,target
0,2021-12-29 23:51:06,161289,879,view,0
1,2021-12-29 23:51:06,120478,1530,view,0


In [136]:
preprocess_posts.dtypes

index                     int64
post_id                   int64
topic                    object
TextCluster               int64
DistanceToCluster_0     float64
DistanceToCluster_1     float64
DistanceToCluster_2     float64
DistanceToCluster_3     float64
DistanceToCluster_4     float64
DistanceToCluster_5     float64
DistanceToCluster_6     float64
DistanceToCluster_7     float64
DistanceToCluster_8     float64
DistanceToCluster_9     float64
DistanceToCluster_10    float64
DistanceToCluster_11    float64
DistanceToCluster_12    float64
DistanceToCluster_13    float64
DistanceToCluster_14    float64
dtype: object

In [210]:
features_data['user_id'].unique()

array([   218,    258,    465, ...,  87670, 124712,  31180],
      shape=(163202,))

In [191]:
user_data['user_id'].unique()

array([   200,    201,    202, ..., 168550, 168551, 168552],
      shape=(163205,))

In [ ]:
"""
        SELECT DISTINCT post_id, user_id, timestamp
        FROM public.feed_data
        WHERE action = 'view'"""